# MPI multiple wrap

Lest start from imports

In [ ]:
import os
from jobqueue_features import (
    MPIEXEC,
    CustomSLURMCluster,
)

from jobqueue_features import mpi_wrap, on_cluster, mpi_task

Let's define task script path.

In [ ]:
script_path = os.path.abspath(
    os.path.join(os.getcwd(), "resources", "helloworld.py")
)

Let's define cluster configuration.

In [ ]:
common_kwargs = {
    "walltime": "00:04:00",
    "cores_per_node": 2,
    "minimum_cores": 2,
    "hyperthreading_factor": 1,
    "ntasks_per_node": 2,
    "memory": "512 MB",
    "mpi_mode": True,
    "env_extra": [
        "export OMPI_ALLOW_RUN_AS_ROOT_CONFIRM=1",
        "export OMPI_ALLOW_RUN_AS_ROOT=1",
    ],
    "mpi_launcher": MPIEXEC,
    "local_directory": "/tmp",
    "queue": "batch",
}

Let's start cluster.

In [ ]:
multi_fork_cluster = CustomSLURMCluster(
    name="multi_fork_cluster",
    fork_mpi=True,
    nodes=1,
    maximum_jobs=2,
    **common_kwargs
)

Let's define task.

In [ ]:
@on_cluster(cluster=multi_fork_cluster)
@mpi_task(cluster_id=multi_fork_cluster.name)
def mpi_wrap_task(**kwargs):
    return mpi_wrap(**kwargs)

First check if we can construct the command:

In [ ]:
result = mpi_wrap_task(
    executable="python",
    exec_args=script_path,
    return_wrapped_command=True
)
result = result.result()
print(result)

Then check the execution of it:

In [ ]:
tasks = []
for x in range(100):
    tasks.append(
        (
            mpi_wrap_task(
                executable="python",
                exec_args=script_path
            )
        )
    )
c1_count = 0
c2_count = 0
for job in tasks:
    result = job.result()["out"]
    if 'c1'.encode() in result:
        c1_count += 1
    elif 'c2'.encode() in result:
        c2_count += 1
print("c1: {} \nc2: {}".format(c1_count, c2_count))

It's alive!